In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [3]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",136016,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
clear_buyers["last_order_date"] = pd.to_datetime(clear_buyers["last_order_date"]) 
clear_buyers = clear_buyers[clear_buyers.last_order_date >= customer_date_desde].copy()
clear_buyers.head()

,customer_id,last_order_date,recency_days,registered_city
0,318928990,2023-03-15,1,BOG
1,318965959,2023-03-15,1,BOG
2,318898052,2023-03-15,1,BOG
3,318653369,2023-03-15,1,SPO
4,318696447,2023-03-15,1,SPO


In [5]:
query = """
WITH
info AS (
SELECT DISTINCT
    s.site_identifier_value as region_code,
    --ffg.close_date,
    bo.customer_id,
    --bs.addl_product_id as padre_sku_id,
    --bs2.name AS card_name,
    --bcat2.name AS category,
    bcat.name AS subcat,
    COUNT(DISTINCT EXTRACT(WEEK FROM bo.submit_date)) AS count_weeks,
    --baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio}) AS gmv_usd
    --SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    --SUM ( boi.quantity* foi.step_unit ) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) >= current_date - interval '10 week' 
   AND DATE(bo.submit_date) <=  current_date
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
   AND bo.customer_id IN {clientes}
GROUP BY 1,2,3--,5,6,7
)

SELECT 
*
FROM info
--WHERE count_weeks > 5
""".format(tipo_cambio=t_cambio, ciudad=city_code, clientes=tuple(clear_buyers.customer_id.unique()))

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [6]:
data_ventas_aux.head()

,region_code,customer_id,subcat,count_weeks,gmv_usd
0,CMX,55603818,Huevos,10,430.96380
1,CMX,242746244,Aceites & Grasas,2,49.85952
2,CMX,71175327,Arroz,2,11.18065
3,CMX,255598575,Verduras,2,25.35616
4,CMX,2668830,Tubérculos,5,17.39996


In [7]:
data_ventas_aux.count_weeks.max()

11

In [8]:
if city_code == 'BOG':
    subcats = ['Azúcar & Endulzantes',  'Aceites & Grasas',  'Arroz',  'Harinas & Mezclas',  'Granos',  'Detergente, Jabón & Lavalozas',  'Papel & Toallas',
            'Desinfectantes & Sanitizantes',  'Cervezas & Licores',  'Pollo Fresco',  'Res Fresco',  'Res Congelado',  'Cerdo Fresco',  'Pescados & Mariscos Congelado', 
            'Alimentos Congelados',  'Contenedores',  'Huevos']
elif city_code == 'BAQ':
    subcats = ['Aceites & Grasas','Alimentos Congelados','Arroz','Azúcar & Endulzantes','Contenedores',
               'Gaseosas','Harinas & Mezclas','Huevos','Pollo Congelado','Tubérculos']
elif city_code == 'SPO':
    subcats = ['Cervejas', 'Refrigerantes e Energéticos', 'Suínos', 'Carne Vermelha', 'Frango', 'Peixes e Frutos do Mar', 'Embutidos', 
            'Batatas Pré-Fritas', 'Legumes', 'Leite', 'Ovos', 'Queijos', 'Detergente, Sabão e Lava-Louças', 'Arroz', 'Café, Chocolates e Infusões', 
            'Feijão', 'Massas e Molhos', 'Azeites, Óleos e Vinagres', 'Frutas']
elif city_code == 'CMX':
    subcats = ['Aceites & Grasas' , 'Arroz' , 'Enlatados' , 'Harinas & Mezclas' , 'Detergente, Jabón & Lavatrastes' , 'Papel & Toallas' , 'Huevos' , 'Leches']
else:
    print("city does not exist yet")

In [9]:
grouped = data_ventas_aux.groupby('subcat').gmv_usd.sum().reset_index()
grouped['weight'] = grouped.gmv_usd / grouped.gmv_usd.sum()
grouped = grouped.sort_values(by=['weight'], ascending=False)
grouped['Cumulative_Weight'] = grouped['weight'].cumsum()
grouped = grouped[(grouped.Cumulative_Weight <= 0.8) & (grouped.subcat.isin(subcats))] 
subcats_exp = grouped.subcat.unique()

In [10]:
subcats_exp =subcats

In [11]:
data_ventas = data_ventas_aux[data_ventas_aux.subcat.isin(subcats_exp)].copy()

ACA HICE UN CAMBIO

In [12]:
f = data_ventas.loc[data_ventas.count_weeks>5].groupby("subcat", as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
f.columns = ['subcat', 'customers', 'list_customers']
f

,subcat,customers,list_customers
0,Aceites & Grasas,590,"[256347402, 251090423, 210338822, 244184848, 6..."
1,Arroz,480,"[244413899, 57436271, 229647712, 15996465, 176..."
2,"Detergente, Jabón & Lavatrastes",229,"[-268847, 239650224, 246834727, 81194401, 4197..."
3,Enlatados,170,"[39109210, -269603, -264753, 23643415, 3627196..."
4,Harinas & Mezclas,526,"[38058519, 58602459, 69872913, -259603, 169568..."
5,Huevos,421,"[55603818, 20948718, 265037282, 28624116, 6319..."
6,Leches,401,"[50901870, 95374212, 174819740, 142937401, 209..."
7,Papel & Toallas,316,"[59338489, 196486540, 152116863, 255567908, -2..."


In [13]:
NO_f = data_ventas.loc[data_ventas.count_weeks<=5].groupby("subcat", as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
NO_f.columns = ['subcat', 'customers', 'list_customers']
NO_f

,subcat,customers,list_customers
0,Aceites & Grasas,3410,"[242746244, -269757, 248715005, 67795488, 1799..."
1,Arroz,3053,"[71175327, 48564584, 96419765, 278438827, 3898..."
2,"Detergente, Jabón & Lavatrastes",3450,"[-268073, 201289315, -269068, -265690, 4066686..."
3,Enlatados,2462,"[209347409, 57441380, 133258425, 159833535, 79..."
4,Harinas & Mezclas,2082,"[165058533, 83665434, 246462758, 16077467, 176..."
5,Huevos,1759,"[-263823, 49312288, 10172246, 266409116, 27262..."
6,Leches,2994,"[52778179, -264857, 246316888, 117363861, 6637..."
7,Papel & Toallas,3394,"[228606071, 38216851, 43731387, 157813952, 174..."


# inicio test MX NO STICKNESS

EN ESTE CASO EL PILOTO DE LOS ELASTICOS VERA EL DESCUENTO A LA MITAD

In [14]:
def create_groups(subcat):
    
    # PRIMERA PARTE PARA ELASTICOS !!!!!!!!!!!!!!!!!!!
    # ...........................................
    # ...........................................
    
    s = NO_f[NO_f.subcat == subcat].list_customers.to_numpy()[0]
    #ACA D SON LOS QUE ENTRAN AL PILOTO
    d = data_ventas[(data_ventas.customer_id.isin(s)) & (data_ventas.subcat == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    #d = data_ventas[(~data_ventas.customer_id.isin(s))].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.30)
    
    alpha=0.01
    aux=0
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(s, test_size = 0.30)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
    # VOY ACAAAAAAAAAA!!!    
    #all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    customer_control_M2 = [int(customer) for customer in customer_control]
    customer_test_M2 = [int(customer) for customer in customer_test]
    
    segt_control_M2 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_CONTROL_MM2", 'customersIds': list(customer_control_M2)}
    segt_test_M2 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_TEST_MM2", 'customersIds': list(customer_test_M2)}
    
    print("M2 INFO SUBCAT:",subcat, "// TEST:",len(customer_test_M2), "// CONTROL:",len(customer_control_M2))
    
    
    
    # SEGUNDA PARTE PARA INELASTICOS !!!!!!!!!!!!!!!!!!!
    # ...........................................
    # ...........................................
    
    s = f[f.subcat == subcat].list_customers.to_numpy()[0]
    #ACA D SON LOS QUE ENTRAN AL PILOTO
    d = data_ventas[(data_ventas.customer_id.isin(s)) & (data_ventas.subcat == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    #d = data_ventas[(~data_ventas.customer_id.isin(s))].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.50)
    
    alpha=0.01
    aux=0
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(s, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
    # VOY ACAAAAAAAAAA!!!    
    #all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    customer_control_M1 = [int(customer) for customer in customer_control]
    customer_test_M1 = [int(customer) for customer in customer_test]
    
    segt_control_M1 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_CONTROL_MM1", 'customersIds': list(customer_control_M1)}
    segt_test_M1 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_TEST_MM1", 'customersIds': list(customer_test_M1)}
    
    print("M1 INFO SUBCAT:",subcat, "// TEST:",len(customer_test_M1), "// CONTROL:",len(customer_control_M1))

    
#ALL --------------------------------------------------------------------------
    all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test_M2)) & (~clear_buyers.customer_id.isin(customer_test_M1)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    #M2
    segt_offer_M2 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_ALL_MM2", 'customersIds': list(all_users)}
    #M1
    segt_offer_M1 = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_ALL_MM1", 'customersIds': list(all_users)}
    
    print('M1 AND M2 ALL: ', len(all_users))
    
    return customer_control_M2, customer_test_M2, segt_test_M2, segt_control_M2, segt_offer_M2, customer_control_M1, customer_test_M1, segt_test_M1, segt_control_M1, segt_offer_M1

TENGO QUE ENTENDER COMO SACAR EL ALL QUE SERIA LA RESTA DE LOS DOS CASOS CON EL TEST YA QUE LOS DOS CONTROLES PUEDEN ESTAR

In [15]:
global_segment_list = []
r= {}

for subcat in f.subcat:
    control_M2, test_M2, segment_t_M2, segment_c_M2, segment_o_M2, control_M1, test_M1, segment_t_M1, segment_c_M1, segment_o_M1 = create_groups(subcat)
    
    #r['CONTROL_'+todays_date+"_"+city_code+"_"+subcat] = control
    #r['TEST_'+todays_date+"_"+city_code+"_"+subcat] = test
    
    global_segment_list.append(segment_t_M1)
    global_segment_list.append(segment_c_M1)
    global_segment_list.append(segment_o_M1)
    global_segment_list.append(segment_t_M2)
    global_segment_list.append(segment_c_M2)
    global_segment_list.append(segment_o_M2)
    
df_test_control = pd.DataFrame.from_dict(r, orient='index').T

M2 INFO SUBCAT: Aceites & Grasas // TEST: 1023 // CONTROL: 2387
M1 INFO SUBCAT: Aceites & Grasas // TEST: 295 // CONTROL: 295
M1 AND M2 ALL:  9980
M2 INFO SUBCAT: Arroz // TEST: 916 // CONTROL: 2137
M1 INFO SUBCAT: Arroz // TEST: 240 // CONTROL: 240
M1 AND M2 ALL:  10144
M2 INFO SUBCAT: Detergente, Jabón & Lavatrastes // TEST: 1035 // CONTROL: 2415
M1 INFO SUBCAT: Detergente, Jabón & Lavatrastes // TEST: 115 // CONTROL: 114
M1 AND M2 ALL:  10147
M2 INFO SUBCAT: Enlatados // TEST: 739 // CONTROL: 1723
M1 INFO SUBCAT: Enlatados // TEST: 85 // CONTROL: 85
M1 AND M2 ALL:  10476
M2 INFO SUBCAT: Harinas & Mezclas // TEST: 625 // CONTROL: 1457
M1 INFO SUBCAT: Harinas & Mezclas // TEST: 263 // CONTROL: 263
M1 AND M2 ALL:  10409
M2 INFO SUBCAT: Huevos // TEST: 528 // CONTROL: 1231
M1 INFO SUBCAT: Huevos // TEST: 211 // CONTROL: 210
M1 AND M2 ALL:  10559
M2 INFO SUBCAT: Leches // TEST: 899 // CONTROL: 2095
M1 INFO SUBCAT: Leches // TEST: 201 // CONTROL: 200
M1 AND M2 ALL:  10196
M2 INFO SUBCAT: 

In [16]:
len(clear_buyers[clear_buyers.registered_city == city_code])

11294

In [17]:
def crear_seg(global_value):
  import requests
  import json

  url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps(global_value)


  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

In [18]:
sgt = []
i=0
for global_seg in global_segment_list:
    rep_text = crear_seg([global_seg])
    sgt.append(rep_text)
    #sgt.append((rep_text[32:].strip('][')[:6], rep_text[32:].strip('][')[38:]))
    #i+=1
    #if i == 3:
    #    break
    time.sleep(20)
sgt

['Customer segments were created: [134801-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_TEST_MM1]',
 'Customer segments were created: [134851-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_CONTROL_MM1]',
 'Customer segments were created: [134655-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_ALL_MM1]',
 'Customer segments were created: [134802-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_TEST_MM2]',
 'Customer segments were created: [134901-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_CONTROL_MM2]',
 'Customer segments were created: [134951-HOOK_Elasticidad2023-03-16_CMX_Aceites & Grasas_ALL_MM2]',
 'Customer segments were created: [135001-HOOK_Elasticidad2023-03-16_CMX_Arroz_TEST_MM1]',
 'Customer segments were created: [134703-HOOK_Elasticidad2023-03-16_CMX_Arroz_CONTROL_MM1]',
 'Customer segments were created: [134852-HOOK_Elasticidad2023-03-16_CMX_Arroz_ALL_MM1]',
 'Customer segments were created: [134853-HOOK_Elasticidad2023-03-16_CMX_Arroz_TEST_MM2]',
 'Customer segment

In [19]:
for x in sgt:
    print(x[32:].strip('][')[:6]+"\t"+x[32:].strip('][')[38:])

134801	Aceites & Grasas_TEST_MM1
134851	Aceites & Grasas_CONTROL_MM1
134655	Aceites & Grasas_ALL_MM1
134802	Aceites & Grasas_TEST_MM2
134901	Aceites & Grasas_CONTROL_MM2
134951	Aceites & Grasas_ALL_MM2
135001	Arroz_TEST_MM1
134703	Arroz_CONTROL_MM1
134852	Arroz_ALL_MM1
134853	Arroz_TEST_MM2
134902	Arroz_CONTROL_MM2
134952	Arroz_ALL_MM2
134854	Detergente, Jabón & Lavatrastes_TEST_MM1
135002	Detergente, Jabón & Lavatrastes_CONTROL_MM1
134803	Detergente, Jabón & Lavatrastes_ALL_MM1
134704	Detergente, Jabón & Lavatrastes_TEST_MM2
134855	Detergente, Jabón & Lavatrastes_CONTROL_MM2
134804	Detergente, Jabón & Lavatrastes_ALL_MM2
135003	Enlatados_TEST_MM1
134903	Enlatados_CONTROL_MM1
134904	Enlatados_ALL_MM1
135051	Enlatados_TEST_MM2
135052	Enlatados_CONTROL_MM2
134905	Enlatados_ALL_MM2
135004	Harinas & Mezclas_TEST_MM1
134906	Harinas & Mezclas_CONTROL_MM1
134805	Harinas & Mezclas_ALL_MM1
134856	Harinas & Mezclas_TEST_MM2
134806	Harinas & Mezclas_CONTROL_MM2
134857	Harinas & Mezclas_ALL_MM2
13

# fin test MX